<a href="https://colab.research.google.com/github/Ihsan-Seker-Git-DS/INM432/blob/main/Lab_1_Word_Counting_with_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Sheet 1: PySpark Demo and Word Counting with Spark

To get you started, we'll walk you through a bit of Colab specific Python and some PySpark code, and then we'll do the classic word count example, followed by some tasks for you to try.

**Please run through the notebook cell by cell (using 'run' above or 'shift-return' on the keyboard).**

##Preliminaries: Preparing Colab and Spark
1.   When you open this notebook from the shared "Big_Data" folder, you **don't have write access**. When you save it, a **copy** will be created in the **folder "Colab Notebooks"**.
2.   The code below will **mount Google Drive** as a **directory** in the file system (the machine is a virtual Linux box). You will need to **authorise** this in a pop-up window that is opened automatically.

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


Next, we check if we can read the `Big_Data` folder. If the command below fails, go back to the shared [`Big_Data`]() folder and click on **"Add to My Drive"** in the folder menu.

In [2]:
%ls "/content/drive/My Drive/Big_Data/data/" 

20_newsgroups/  hamlet.txt  lingspam_public/  spark/
foodhygiene/    library/    movielens-small/  stopwords2.txt


Next, we **install Spark** (may take a minute or two). This will need to be done **every time a new machine is created**. 


In [3]:
%cd
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # installing java
!tar -xzf "/content/drive/My Drive/Big_Data/data/spark/spark-3.2.0-bin-hadoop2.7.tgz" # unpacking 
import os # Python package for interaction with the operating system
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" # tell the system where Java lives
os.environ["SPARK_HOME"] = "/root/spark-3.2.0-bin-hadoop2.7" # and where spark lives
!pip install -q findspark # install helper package
import findspark # use the helper package
findspark.init() # to set up spark
%cd "/content/drive/My Drive/Big_Data" 

/root
/content/drive/.shortcut-targets-by-id/10ijQ0eE11e3MwBmUKbK5TXTdvKIqsjXZ/Big_Data


## Part 1 - Demo: Apache Spark API with PySpark

Basically there are **2 APIs** available in Apache Spark - **RDD** (Resilient Distributed Datasets) and **DataFrame** (extended by Dataset in Scala and Java). In this lab we will look at RDDs and Dataframes in Python. 

For **more information** on the **Spark framework** - visit (https://spark.apache.org).
For more information on the **Pyspark API** - visit (https://spark.apache.org/docs/latest/api/python/index.html).

### 1) Access to Spark

We start by cretaing a **SparkContext**, normally called **`sc`**. 
We use that to create RDDs and a **SparkSession** object (for DataFrames), often just called **`spark`**. 

In [4]:
import pyspark
# get a spark context
sc = pyspark.SparkContext.getOrCreate() # What is spark context?
print(sc)
# get the context
spark = pyspark.sql.SparkSession.builder.getOrCreate() # What is spark session?
print(spark) 

<SparkContext master=local[*] appName=pyspark-shell>


### 2) RDD Creation

There are **multiple** ways to **create RDDs**. 

One is to **parallelise** a **Python object** that exists in your driver process (i.e. the process that runs this notebook), which we will do in the next cell. 

Anoter way is to create an **RDD** is by referencing an **external dataset** such as a shared filesystem, HDFS, HBase, or just data source offering a Hadoop InputFormat. This is what we will be using later in this lab (further down).

In [6]:
# Create an RDD from a Python object in this process (the "driver").
# The parallelize function  creating the "numbers" RDD
data = [1,2,3,4,5]
firstRDD = sc.parallelize(data) # Parallelize function allows Spark to distribute the data across multiple nodes (up to 100 linux vm's), instead of depending on a single node to process the data
print(firstRDD)
print(firstRDD.getStorageLevel())
print(firstRDD.getNumPartitions())

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274
Serialized 1x Replicated
2


This RDD lives now on as many worker machines as are available and as are deemed useful by Spark. 

We can see that **`firstRDD`** has only been replicated once, which is because we only have a single machine running Spark. There are 2 partitions, so we can have 2 parallel threads 

### 3) RDD operations
RDDs have two kinds of operations: ***Transformations*** and ***Actions***.

***Transformations*** create a new RDD by applying a function to the items in the RDD. The function will be registered with the new RDD, but only be applied when needed. This is called ***lazy evaluation***.

***Actions*** produce some output from the data. An *Action* will trigger the execution of all *Transformations* registered with the RDD.

Here are some examples:

In [7]:
def myfun(x):
  return x+3
# lambda function: x -> x+3
# RDD2 = firstRDD.map(lambda x:x+3)
RDD2 = firstRDD.map(myfun)  
print(RDD2)
# nothing happened to far, as there is no action

PythonRDD[1] at RDD at PythonRDD.scala:53


If the functions are short (one expression, to be exact), it can be more convenient to write a **lambda** expression, that creates an **anonymous function**. 

In [8]:
RDD3 = firstRDD.map(lambda x:x+3) # this is the same as using myfun 
print(RDD3)
# nothing happened to far, as there is no action

PythonRDD[2] at RDD at PythonRDD.scala:53


So far, the function we created with the lambda has not actually been executed. 

Next we use **`collect`**, which is an **action** and therefore triggers the execution. 
**`collect`**returns the items in the RDD back into this local driver process in a Python array.](https://)

In [9]:
a = RDD2.collect() 
print(a)

b = RDD3.collect() 
print(b)

[4, 5, 6, 7, 8]
[4, 5, 6, 7, 8]


As we can seee above, *myfun* (RDD2) and the *lambda x: x+3* (RDD3) have the same effect.

Look here for more information about the functions provided by the RDD class: (https://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD). 

### 4) Dataframes 

**Dataframes** are a **more structured** form of storage than RDDs and similar to Pandas dataframes.  

Let us see how to create and use dataframes. There are three ways of **creating a dataframe**:
1. from an existing RDD.
2. form and external data source, e.g., loading the data from JSON or CSV files.
3. programmatically specifying schema and data. 

Here is an example for option a). We use the *Row* class to create structured data rows.

In [10]:
from pyspark.sql import Row

dataList = [('Anne',21),('Bob',22),('Carl',29),('Daisy',36)] # our data as a list
rdd = sc.parallelize(dataList) # RDD from the list
peopleRDD = rdd.map(lambda x: Row(name=x[0], age=int(x[1]))) # RDD
peopleDF = spark.createDataFrame(peopleRDD) 
print(peopleDF)
peopleDF.show()

DataFrame[name: string, age: bigint]
+-----+---+
| name|age|
+-----+---+
| Anne| 21|
|  Bob| 22|
| Carl| 29|
|Daisy| 36|
+-----+---+



## Part 2: Classic Word Count

We will now do the **classic word count example** for the MapReduce pattern.

We will apply it to the text of Sheakespeare's play *Hamlet*. To access that, you should have added the `Big_Data` folder to your own Google Drive. Double check that this part of the **preliminaries** section above was executed correctly, otherwise go back and fix this first.

### 1) Load the data
First we need to load the **text into an RDD** (the second method of creating an RDD as mentioned above). 

We need to specify the path, and we can read directly from the shared Big_Data directory. 

In [11]:
filepath = "/content/drive/My Drive/Big_Data/data/hamlet.txt"

You can read the file into an RDD with **`textFile`**. The RDD then contains as items the **lines of the text**. **`take(3)`** then gives us the first 3 lines.  

In [12]:
lineRDD = sc.textFile(filepath)
lineRDD.take(3) # Take is an action which takes the n number of lines specified, 3 in this case. 

['Project Gutenberg Etext of Hamlet by Shakespeare',
 "PG has multiple editions of William Shakespeare's Complete Works",
 '']

### 2) Split lines into words

In order to count the words, we need to split the lines into words. We can do that using the **`split`** function of the Python String class to separate at each space. 

The map function replaces each item with a new one. In this case, our **`lambda`** returns an array of words (provided by `split(' ')`). However, we want to create one item per word. Therefore, we need to use a function called **`flatMap`** that creates a new RDD item for every item in the array returned by the lambda.  

In [13]:
wordRDD = lineRDD.flatMap(lambda x: x.split(' '))
wordRDD.take(3)

['Project', 'Gutenberg', 'Etext']

We  then **map** the words to **tuples** of the form *(word, 1)*.

In [14]:
word1RDD = wordRDD.map(lambda x: (x, 1))
word1RDD.take(3)

[('Project', 1), ('Gutenberg', 1), ('Etext', 1)]

### 3) Count by reducing
For Spark, the first part in each tuple is the '**key**'. Now we can use **`reduceByKey()`** to add the 1s, giving the number of occurences per word.

In [15]:
wordCountRDD = word1RDD.reduceByKey(lambda x,y: x+y )
wordCountRDD.take(3)

[('Project', 21), ('Etext', 4), ('of', 679)]

### 4) Filtering 

There are many empty strings returned by the splitting. We can remove them by **filtering**.
**`filter()`** keeps only the items where the test function (specified by our lambda) returns `true`.

In [16]:
wordFilteredRDD = wordRDD.filter(lambda x: len(x)>0)
word1RDD = wordFilteredRDD.map(lambda x: (x, 1))
wordCountRDD = word1RDD.reduceByKey(lambda x,y: x+y )
wcList = wordCountRDD.collect()
print(wcList[1:5])

[('Etext', 4), ('of', 679), ('Shakespeare', 5), ('multiple', 3)]


## Part 3: Tasks for you to work on

Based on the examples above, you can now try and write some code yourself.  Look for the lines starting with **>>>**. You neeed to fix them by writing your own code.

### 0) Warm-up: Take a shortcut

We can take a shortcut and use a **ready-made function** **`countByValue()`**, which does the same as we did above without us specifying a lambda for addition (https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.countByValue).

In [21]:
wordFilteredRDD = wordRDD.filter(lambda x: len(x)>0)
wcList =  wordFilteredRDD.countByValue()# your code goes here, use countByValue on wordFilteredRDD to create wcList.
print(wcList)

defaultdict(<class 'int'>, {'Project': 21, 'Gutenberg': 16, 'Etext': 4, 'of': 679, 'Hamlet': 31, 'by': 118, 'Shakespeare': 5, 'PG': 2, 'has': 17, 'multiple': 3, 'editions': 2, 'William': 4, "Shakespeare's": 2, 'Complete': 2, 'Works': 2, 'Copyright': 1, 'laws': 2, 'are': 127, 'changing': 1, 'all': 94, 'over': 4, 'the': 1044, 'world,': 3, 'be': 198, 'sure': 5, 'to': 648, 'check': 2, 'copyright': 8, 'for': 185, 'your': 235, 'country': 2, 'before': 14, 'posting': 1, 'these': 42, 'files!!': 1, 'Please': 2, 'take': 29, 'a': 478, 'look': 22, 'at': 85, 'important': 2, 'information': 4, 'in': 398, 'this': 230, 'header.': 1, 'We': 36, 'encourage': 1, 'you': 401, 'keep': 18, 'file': 4, 'on': 111, 'own': 36, 'disk,': 2, 'keeping': 1, 'an': 53, 'electronic': 1, 'path': 2, 'open': 3, 'next': 5, 'readers.': 2, 'Do': 23, 'not': 260, 'remove': 1, 'this.': 6, '**Welcome': 1, 'To': 121, 'The': 155, 'World': 1, 'Free': 2, 'Plain': 1, 'Vanilla': 1, 'Electronic': 1, 'Texts**': 1, '**Etexts': 1, 'Readable': 

### 1) Better splitting 

Currently our 'words' can contain punctuation, because only spaces are removed. A better way to split is using regular expressions  [Python's 're' package](https://docs.python.org/3.5/library/re.html?highlight=regular%20expressions). **`re.split('\W+', 'my. test. string!')`** does a good job. Try it out below by fixing the line that starts with '>>>'.

In [22]:
import re
wordRDD = lineRDD.flatMap(lambda x: re.split('\W+', 'my. test. string!') ) # apply re.split('\W+', string) here
wordFilteredRDD = wordRDD.filter(lambda x: len(x)>0) # filtering
wordFilteredRDD.take(3)

['my', 'test', 'string']

### 2) Use lower case

**Convert all strings** to lower case (using **`.lower()`** provided by the Python string class), so that 'Test' and 'test' count as the same word. Package it into one a tuple of the form (word,1) in the same call.

In [28]:
wordLowerRDD = wordFilteredRDD.map(lambda x: x.lower())
wordLowerRDD.take(3)

['my', 'test', 'string']

In [29]:
word1LowerRDD = wordLowerRDD.map(lambda x: (x, 1))
wordCountLowerRDD = word1LowerRDD.reduceByKey(lambda x,y: x+y) # we can now get better word count results
wordCountLowerRDD.take(3)

[('test', 7066), ('my', 7066), ('string', 7066)]

### 3) Filter rare words

Add a filtering step call **remove all words with less than 5 occurrences**. This can be useful to if you want to identify common topics in documents, where very rare words can be misleading. 

In [35]:
# the trick here is to apply the lambda only to the second part of each item, i.e. x[1] 
freqWordsRDD = wordCountRDD.filter(lambda x: len(x)<5 ) # tip: filter keeps the elements for which the lambda function returns true
freqWordsRDD.take(3)

[('Project', 21), ('Etext', 4), ('of', 679)]

### 4) List only stopwords

**Stopwords** are frequent words that are not topic-specifc (articles, pronouns, ...). Removing stopwords can be useful in recognising the topic of a document. Removing non-stopwords can be useful for recognising the style of an author. 

Below is a small list of stopwords. Filter so that you keep the tuples where the **first element is in the stopword list**.

In [36]:
stopWordList = ['the','a','in','of','on','at','for','by','I','you','me'] 
stopWordsRDD = freqWordsRDD.filter(lambda x: x[0] in stopWordList  ) # the 1st part of the tuple should be in the list 

We can now list the results (there are only a few words. 

In [37]:
output = stopWordsRDD.collect() 
for (word, count) in output:
    print("%s: %i" % (word, count))

of: 679
at: 85
in: 398
by: 118
the: 1044
for: 185
a: 478
you: 401
on: 111
I: 511
me: 142


## Reading

Read chapter 1 of Lescovec et al (2019), "Mining of Massive Datasets", and work out the answers to exercise 1.2.1 on page 7 and 1.3.1 and 1.3.2 on page 15. If you have time, start reading chapter 2.

## Spark @home (optional)

You can try and install Spark on your own laptop or desktop, using the instructions provided on Moodle.